In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the current device index (default is 0 if no other device is specified)
    current_device = torch.cuda.current_device()
    
    # Get the name of the GPU at this device index
    gpu_name = torch.cuda.get_device_name(current_device)
    print(f"Current GPU: {gpu_name}")
else:
    print("No GPU available.")

Current GPU: Tesla P40


In [2]:
# !huggingface-cli download dataset longquan/llm-japanese-dataset-split_10
from datasets import load_dataset

# Make sure to provide the correct `repo_id`
dataset = load_dataset("longquan/llm-japanese-dataset-split_10")

In [3]:
# Load dataset from local cache
dataset = load_dataset("longquan/llm-japanese-dataset-split_10", cache_dir="~/.cache/huggingface/datasets")

# View available dataset splits
print("Available Splits:", dataset.keys())

# Load specific split (e.g., 'train') and inspect the first few rows
train_data = dataset["train"]
print("First Rows of Train Split:", train_data[0])

Available Splits: dict_keys(['train'])
First Rows of Train Split: {'instruction': '次の日本語には間違っている部分があります。その部分を直して、正しい日本語の文を出力してください。', 'input': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとし２０１４年１１月より開始された事前録音フロート番組。', 'output': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとして２０１４年１１月より開始された事前録音フロート番組。'}


In [4]:
from transformers import pipeline

In [6]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [10]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)


messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please summarize the following text: 'Artificial intelligence is a field of computer science that enables machines to perform tasks that typically require human intelligence.'"}
]

messages


[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': "Please summarize the following text: 'Artificial intelligence is a field of computer science that enables machines to perform tasks that typically require human intelligence.'"}]

In [17]:
outputs = pipe(
    messages,
    max_new_tokens=128,
)
print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'The text states that artificial intelligence (AI) is a field of computer science that allows machines to perform tasks that are typically done by humans, using algorithms and data.'}


In [16]:
#Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
outputs = pipe(
    messages,
    max_new_tokens=128,
    pad_token_id = 128001
)
print(outputs[0]["generated_text"][-1])


{'role': 'assistant', 'content': 'The text states that artificial intelligence (AI) is a field of computer science that enables machines to perform tasks that are typically done by humans.'}


In [18]:
sample = train_data[0]
sample['output']

'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとして２０１４年１１月より開始された事前録音フロート番組。'

In [19]:
messages = [
    {"role": "system", "content": sample["instruction"]},
    {"role": "user", "content": sample["input"]}
]
messages

[{'role': 'system',
  'content': '次の日本語には間違っている部分があります。その部分を直して、正しい日本語の文を出力してください。'},
 {'role': 'user',
  'content': 'フライデースペシャル』（ニッポン放送）内で、１４：４０頃から放送されていた生放送コーナー「森野熊八\u3000体いきいき楽しい食卓」が２０１４年１０月にて終了となり、後コーナーとし２０１４年１１月より開始された事前録音フロート番組。'}]

In [22]:


outputs = pipe(
    messages,
    max_new_tokens=512,
)
print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '「森野熊八\u3000体いきいき楽しい食卓」は、ニッポン放送（日本放送協会）内で、14：40頃から放送されていた生放送コーナーで、２０１４年１０月に終了した。後に、２０１４年１１月より事前録音フロート番組として始まった。'}


In [23]:
import random

In [28]:
random_sample = train_data[random.choice(range(train_data.num_rows))]

print("Reference: ",random_sample['output'],"\n")

messages = [
    {"role": "system", "content": random_sample["instruction"]},
    {"role": "user", "content": random_sample["input"]}
]
outputs = pipe(
    messages,
    max_new_tokens=128,
)
print("Model: ", outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Reference:  ２００６年　−　品質保証マネジメントシステム、情報セキュリティマネジメントシステム、プライバシーマークの認証を取得 

Model:  {'role': 'assistant', 'content': '２００６年\u3000−\u3000品質保証マネジメントシステム、情報セキュリテイマネジメントシステム、プライバシーマークの認定を取得'}


### References

https://huggingface.co/datasets/longquan/llm-japanese-dataset-split_10